In [1]:
import ray

ray.init(ignore_reinit_error=True)

2023-12-11 11:44:32,689	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.11.4
Ray version:,2.8.1


In [2]:
ray.__version__

'2.8.1'

In [3]:
from ray import train, tune
from fastcore.xtras import Path

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm

from ray.tune.logger import pretty_print

2023-12-11 11:44:34,543	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.


In [5]:
import torch
import os
# Set device currently does nothing
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
if "mps" == device: os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = "1"

In [6]:
#!pip install -e aerospaceRL

In [7]:
import sys
sys.path.append(str(Path('.').resolve()/'aerospaceRL'))
import aero_gym

In [8]:
#no GPU support. Still an open issue:
#https://github.com/ray-project/ray/issues/28321
#tune.Tuner(tune.with_resources(0, resources={"cpu": 8, "gpu": 1}),...
#https://docs.ray.io/en/latest/tune/api/doc/ray.tune.with_resources.html
ray.get_gpu_ids()

[]

In [9]:
#grid search parameters
# tune.grid_search([0.01, 0.001])


In [10]:
#custom model example: https://docs.ray.io/en/latest/rllib/rllib-models.html

In [11]:
from aero_gym.envs import dubins_aircraft#.DubinsAircraft

In [12]:
#aerospaceRL.aero_gym.envs.dubins_aircraft.DubinsAircraft

In [13]:
#customEnv=dubins_aircraft.DubinsAircraft()

In [14]:
tune.registry.register_env("myEnv", dubins_aircraft.DubinsAircraft)

In [15]:
# agent, model, and environment setup
param_space = (PPOConfig()
            .environment(env="myEnv")
            .framework("torch")
            .training(lr=0.01, model=dict(fcnet_activation='relu')))

# sets save directory and when to end training                        
run_config=train.RunConfig(name="my_tune_customEnv",
                           storage_path=str(Path('.').resolve()/'saved_agents'),
                           stop={"episode_reward_mean": 100},
                           checkpoint_config=train.CheckpointConfig(
                                            checkpoint_score_attribute="episode_reward_mean",
                                            checkpoint_score_order="max"
                                                                   ))
#load configs into Tuner
tuner = tune.Tuner("PPO", run_config=run_config,
                   param_space=param_space,
                    )
#training model
results = tuner.fit()


(pid=11122) 2023-12-11 11:44:39,876	WARNING __init__.py:10 -- PG has/have been moved to `rllib_contrib` and will no longer be maintained by the RLlib team. You can still use it/them normally inside RLlib util Ray 2.8, but from Ray 2.9 on, all `rllib_contrib` algorithms will no longer be part of the core repo, and will therefore have to be installed separately with pinned dependencies for e.g. ray[rllib] and other packages! See https://github.com/ray-project/ray/tree/master/rllib_contrib#rllib-contrib for more information on the RLlib contrib effort.
2023-12-11 11:44:42,303	ERROR tune_controller.py:1383 -- Trial task failed for trial PPO_myEnv_92c0d_00000
Traceback (most recent call last):
  File "/Users/sbrewer/anaconda3/envs/RLlib/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/Users/sbrewer/anaconda3/envs/RLlib/lib/python3.11/site-packages/ray/_private/auto_init_

2023-12-11 11:44:42,310	WARNING experiment_state.py:327 -- Experiment checkpoint syncing has been triggered multiple times in the last 30.0 seconds. A sync will be triggered whenever a trial has checkpointed more than `num_to_keep` times since last sync or if 300 seconds have passed since last sync. If you have set `num_to_keep` in your `CheckpointConfig`, consider increasing the checkpoint frequency or keeping more checkpoints. You can supress this warning by changing the `TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S` environment variable.
2023-12-11 11:44:42,312	ERROR tune.py:1043 -- Trials did not complete: [PPO_myEnv_92c0d_00000]
2023-12-11 11:44:42,312	INFO tune.py:1047 -- Total run time: 4.08 seconds (4.06 seconds for the tuning loop).
2023-12-11 11:44:42,314	WARNING experiment_analysis.py:185 -- Failed to fetch metrics for 1 trial(s):
- PPO_myEnv_92c0d_00000: FileNotFoundError('Could not fetch metrics for PPO_myEnv_92c0d_00000: both result.json and progress.csv wer

In [ ]:
# Find best model
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint


algo = Algorithm.from_checkpoint(best_checkpoint.path)

In [ ]:
best_checkpoint.path

In [ ]:
policy = algo.get_policy()
#print(policy.get_weights())
model = policy.model

In [ ]:
model

In [ ]:
# render agent
import gym
import matplotlib.pyplot as plt
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output

N=500
env_name = "CartPole-v1"
env = gym.make(env_name, render_mode="rgb_array")

reward_lst=[]
for n in range(N):
    episode_reward = 0
    obs, info = env.reset()

    while True:

        action = algo.compute_single_action(obs)
        obs, reward, done, truncated, info = env.step(action)
        episode_reward += reward
        if n ==N-1:
            clear_output(wait=True)
            plt.imshow( env.render())
            plt.title(f'Reward: {int(episode_reward)}')
            plt.show()

        if done or episode_reward>500:
            reward_lst.append(episode_reward)
            break

env.close()

In [ ]:
import pandas as pd
pd.Series(reward_lst).describe()

In [ ]:
list(gym.envs.registry.keys())

In [ ]:
import datetime as dt
end=dt.datetime.now()
print(f'Finished: {end.strftime("%A %B %d, %Y")} at {end.strftime("%H:%M")}')